In [2]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [3]:
from langchain_community.document_loaders import PyPDFLoader
import pandas as pd
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
load_dotenv(dotenv_path='../.envrc')
embedding_function = OpenAIEmbeddings(model="text-embedding-3-large")


In [3]:
df = pd.read_csv('db_links.csv')
df

,Unnamed: 0,orig,pdf
0,0,https://papers.nips.cc/paper/4824-imagenet-cla...,https://papers.nips.cc/paper/4824-imagenet-cla...
1,1,https://arxiv.org/abs/1406.2661,https://arxiv.org/pdf/1406.2661
2,2,https://arxiv.org/abs/1409.1556,https://arxiv.org/pdf/1409.1556
3,3,https://arxiv.org/abs/1512.03385,https://arxiv.org/pdf/1512.03385
4,4,https://arxiv.org/abs/1409.4842,https://arxiv.org/pdf/1409.4842
...,...,...,...
99,99,https://arxiv.org/pdf/1410.5401.pdf,https://arxiv.org/pdf/1410.5401.pdf
100,100,https://arxiv.org/pdf/1512.02595.pdf,https://arxiv.org/pdf/1512.02595.pdf
101,101,https://arxiv.org/pdf/2001.08361.pdf,https://arxiv.org/pdf/2001.08361.pdf
102,102,https://arxiv.org/pdf/math/0406077.pdf,https://arxiv.org/pdf/math/0406077.pdf


In [4]:
pages = []
for i in range(0, len(df)):
    try:
        loader = PyPDFLoader(df.loc[i, 'pdf'])
        pages.extend(loader.load_and_split())
    except:
        continue

invalid pdf header: b'\x89PNG\r'
EOF marker not found
EOF marker not found
EOF marker not found
Ignoring wrong pointing object 2 65536 (offset 0)
Ignoring wrong pointing object 34 65536 (offset 0)
Ignoring wrong pointing object 92 65536 (offset 0)
Ignoring wrong pointing object 145 65536 (offset 0)
Ignoring wrong pointing object 206 65536 (offset 0)
Ignoring wrong pointing object 274 65536 (offset 0)
Ignoring wrong pointing object 330 65536 (offset 0)
Ignoring wrong pointing object 372 65536 (offset 0)
invalid pdf header: b'<!DOC'
EOF marker not found


In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2500,
    chunk_overlap=20,
    length_function=len,
)

In [6]:
docs = []
for doc in pages:
    temp = text_splitter.create_documents(doc.page_content)
    for t in temp:
        t.metadata['source'] = doc.metadata['source']
        t.metadata['page'] = doc.metadata['page']
    docs.extend(temp)

In [7]:
db = Chroma.from_documents(pages, embedding_function, persist_directory="paper_info")

: 

In [4]:
db = Chroma(persist_directory="paper_info", embedding_function=embedding_function)
retriever = db.as_retriever(search_kwargs={'k': 5})

In [5]:
from openai import OpenAI
client = OpenAI()

In [7]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [25]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a helpful deep learning mentor. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: 
    {question}
    CONTEXT: 
    {context}
    """.strip()

    context = format_docs(search_results)
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

def search(query):

    results = retriever.invoke(query)
    return results

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer


In [24]:
rag("What is the main idea of attention mechanism?")

'The main idea of the attention mechanism is to dynamically focus on different parts of the input data when producing the output. This approach assigns weights to different input values based on a compatibility function between a query and corresponding keys, allowing the model to emphasize relevant information and ignore less pertinent information at each step of the output generation.\n\nFor example, in image captioning, the attention mechanism allows the model to "attend" to various regions of an image while generating the corresponding textual description. Similarly, in natural language processing tasks like translation, it helps the model to focus on relevant words in the source sentence when generating each word in the target sentence. Scaled Dot-Product Attention and Multi-Head Attention are specific implementations that enhance the capability to attend to different parts of the input more effectively and in parallel, respectively. The former scales the dot product by the dimens